### All imports

In [1]:
# Data processing
import pandas as pd
import numpy as np
import csv

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns


#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

# Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
user_item_rating = pd.read_csv("user-item-rating.csv",  on_bad_lines='skip', sep = '\t', names = ['user_id', 'item_id', 'rating'])
user_item_rating


,user_id,item_id,rating
0,455,50,3.0
1,455,457,4.0
2,455,28,5.0
3,455,458,3.0
4,455,459,5.0
...,...,...,...
50676,84839,131,3.0
50677,84839,109,5.0
50678,84839,145,5.0
50679,84839,133,5.0


In [3]:
user_item_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50681 entries, 0 to 50680
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  50681 non-null  int64  
 1   item_id  50681 non-null  int64  
 2   rating   50681 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.2 MB


In [4]:
# Checking for any missing data

print("Percentage null or na values in Dataset\n-------------------------------------")
((user_item_rating.isnull() | user_item_rating.isna()).sum() * 100 / user_item_rating.index.size).round(2)

Percentage null or na values in Dataset
-------------------------------------


user_id    0.0
item_id    0.0
rating     0.0
dtype: float64

In [5]:
######    IKKE VIKTIG MEN SER FINT UT???      ######


# Number of users, recipies and ratings
print('The number of users in the dataset:', user_item_rating['user_id'].nunique())

print('The number of recipies that are rated:', user_item_rating['item_id'].nunique())

print('The number of different ratings in the dataset:', user_item_rating['rating'].nunique())

print('The unique ratings are:', sorted(user_item_rating['rating'].unique()))

The number of users in the dataset: 1273
The number of recipies that are rated: 1031
The number of different ratings in the dataset: 5
The unique ratings are: [1.0, 2.0, 3.0, 4.0, 5.0]


In [6]:
# reading in data from "item-profiles2.csv" to get the recipe names

item_profiles2 = pd.read_csv("item-profiles2.csv",  on_bad_lines='skip', sep = ';')

item_profiles2

,Recipe ID,Name,Fiber (g),Sodium (g),Carbohydrates (g),Fat (g),Protein (g),Sugar (g),Saturated Fat (g),Size (g),Servings,Calories (kCal),Average Rating,Average Sentiment,Number of Ratings,Number of Bookmarks,Year of Publishing
0,2622,Slow Cooker Tender and Yummy Round Steak,4.5,0.83,33.1,13.6,33.8,5.0,4.6,2599.35,6.0,393.0,4.32,1.79,81,2271,2000
1,722,Chicken Pot Pie II,6.2,1.06,47.8,29.5,51.4,6.4,11.4,2137.86,4.0,666.0,4.66,2.02,116,1200,2000
2,1137,Chicken in a Pot,1.0,0.40,6.9,6.6,28.7,1.8,1.4,819.37,4.0,206.0,4.29,1.98,83,1779,2001
3,2502,Erin's Indonesian Chicken,6.4,0.32,58.1,18.6,35.4,7.8,3.8,1972.13,4.0,530.0,4.39,1.99,80,872,2005
4,2714,Bubble Pizza,3.0,1.96,45.4,36.4,28.5,8.7,13.4,2375.00,8.0,624.0,4.35,1.96,117,2204,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,1353,Beef and Biscuit,0.9,0.63,15.5,18.9,18.4,3.7,8.6,1536.45,10.0,306.0,3.99,1.52,79,811,2001
1027,987,Creamy Pesto Shrimp,2.7,0.44,43.0,42.5,23.1,0.2,24.3,1655.88,8.0,646.0,4.58,1.98,225,3134,2000
1028,2903,Boiled Chicken,1.2,0.06,4.5,11.1,16.3,1.9,3.0,1780.00,8.0,186.0,4.70,1.20,74,1083,2001
1029,2136,Mushroom Sauce Baked Pork Chops,2.5,1.11,28.1,14.3,19.0,6.5,6.1,2241.75,6.0,316.0,4.45,1.86,121,1188,2001


The dataframe shown above has several columns that are of no use to us, we only want the columns "Recipe ID" and "Name". 

In [7]:
recipe_names = item_profiles2[['Recipe ID', 'Name']].copy()
recipe_names.head()

,Recipe ID,Name
0,2622,Slow Cooker Tender and Yummy Round Steak
1,722,Chicken Pot Pie II
2,1137,Chicken in a Pot
3,2502,Erin's Indonesian Chicken
4,2714,Bubble Pizza


Now that we have extracted only the preferred columns, we can merge this dataframe with "user_item_rating", our other dataframe that contains the user id, recipe id and ratings. To merge these, we use "Recipe ID". Before we merge them, we change the name of the column "Recipe ID" to "item_id" to match the column in "user_item_rating". 

In [8]:
recipe_names.rename(columns={'Recipe ID':'item_id'}, inplace=True)
recipe_names.head()


,item_id,Name
0,2622,Slow Cooker Tender and Yummy Round Steak
1,722,Chicken Pot Pie II
2,1137,Chicken in a Pot
3,2502,Erin's Indonesian Chicken
4,2714,Bubble Pizza


In [9]:
df = user_item_rating.merge(recipe_names, how= 'inner', on='item_id')
df.head()


,user_id,item_id,rating,Name
0,455,50,3.0,Baked Ziti I
1,2878,50,2.0,Baked Ziti I
2,3172,50,5.0,Baked Ziti I
3,3698,50,5.0,Baked Ziti I
4,3794,50,5.0,Baked Ziti I


In [10]:
# We want to check if any recipes have very few ratings, or if any 
# users have given very few ratings. We also find the mean. 

# Group data by user, count number of ratings and shows the mean for each user: 
aggregate_by_user = df.groupby('user_id').agg(mean_rating = ('rating', 'mean'), 
number_of_ratings = ('rating', 'count')).reset_index()



# Sorts the dataframe by number of ratings, helps us see what the lowest number of recipes any user has rated is
# This also shows us that there are 1273 unique users that have given ratings (number of rows)

aggregate_by_user.sort_values('number_of_ratings')


,user_id,mean_rating,number_of_ratings
260,22109,4.550000,20
518,41530,4.600000,20
1085,75525,4.450000,20
1082,75380,5.000000,20
760,57892,3.950000,20
...,...,...,...
597,48719,4.750000,236
1142,78714,4.135802,243
320,25694,4.657258,248
1094,76151,3.771331,293


In [11]:

# Group data by recipe, count number of ratings and shows mean rating for each recipe:
aggregate_by_recipe = df.groupby('item_id').agg(mean_rating = ('rating', 'mean'), 
number_of_ratings = ('rating', 'count')).reset_index()


# Sorts the dataframe by number of ratings, shows us the number of times the least rated recipe has been rated.
#       By not using .head(), we can also see the recipes with the highest number of ratings. 
# This also shows us that there are 1031 unique recipes (number of rows)

aggregate_by_recipe.sort_values('number_of_ratings')

,item_id,mean_rating,number_of_ratings
1030,15746,4.450000,20
654,1976,4.600000,20
657,2006,4.100000,20
680,2118,4.500000,20
685,2152,4.450000,20
...,...,...,...
64,134,4.600000,290
68,140,4.429553,291
21,50,4.594156,308
66,137,4.651090,321


Seeing that no recipe is rated less than 20 times, and none of our users have given less than 20 ratings, we decide that there is no reason to remove any data. 

In [12]:
# Shows the recipes that has the best mean - given best ratings - at the top,
#   the recipes with the lowest mean - worse ratings - at the bottom:
aggregate_by_recipe.sort_values('mean_rating', ascending=False)

,item_id,mean_rating,number_of_ratings
397,956,4.960000,25
245,572,4.954545,22
70,143,4.887097,62
842,3437,4.882353,34
704,2312,4.869565,23
...,...,...,...
168,372,3.583333,36
483,1247,3.553846,65
727,2534,3.428571,28
448,1108,3.393939,33


In [13]:
# NOT IMPORTANT, JUST SHOWS THE DIFFERENT MEANS

df_mean_users = aggregate_by_user['mean_rating'].mean()
df_ratings_mean = df['rating'].mean()
df_mean_items = aggregate_by_recipe['mean_rating'].mean()
print('Mean of all the users own means: ',df_mean_users, '\n' 'Mean of ratings from "normal" df:', df_ratings_mean,'\n' 'Mean of each recipes own mean:', df_mean_items)

Mean of all the users own means:  4.404332547821255 
Mean of ratings from "normal" df: 4.388054695053373 
Mean of each recipes own mean: 4.352838067432242


### Følger en tutorial!
De gjør ting annerledes enn vi hadde originalt, vil bare teste litt forskjellig

In [14]:
# BUILDING RATINGS MATRIX

#Can use either item_id or Name as column, same thing?
ratings_matrix = df.pivot_table(values='rating', index='user_id', columns='Name')

ratings_matrix.head()

Name,A Good Easy Garlic Chicken,A Jerky Chicken,Acapulco Chicken,Actually Delicious Turkey Burgers,Addictive Sweet Potato Burritos,African Chicken Stew,Aimee's Quick Chicken,Alaska Salmon Bake with Pecan Crunch Coating,Alfredo Mostaccioli,Alice Chicken,...,Whole Wheat and Honey Pizza Dough,World's Best Lasagna,Yakisoba Chicken,Yummy Honey Chicken Kabobs,Yummy Pork Chops,Zesty Slow Cooker Chicken Barbecue,Zippy Summer Shrimp,Zucchini Alfredo,Zucchini Parmesan,Zucchini Patties
user_id,,,,,,,,,,,,,,,,,,,,,
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
1155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
1299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Some people give generally higher - or lower - ratings than others, and have different views on what a "bad rating" is (For some poeple a rating of 3 is average or "just fine", while others would consider 3 a horrendous rating). Because of this we have to normalize our data by extracting each users average rating. Movies with a rating less than the users average will get a negative rating, and movies with a rating higher than the users average will get a positive value. 

In [15]:
# Normalizing the ratings matrix

norm_ratings_matrix = ratings_matrix.subtract(ratings_matrix.mean(axis=1), axis='rows')

norm_ratings_matrix.head()

Name,A Good Easy Garlic Chicken,A Jerky Chicken,Acapulco Chicken,Actually Delicious Turkey Burgers,Addictive Sweet Potato Burritos,African Chicken Stew,Aimee's Quick Chicken,Alaska Salmon Bake with Pecan Crunch Coating,Alfredo Mostaccioli,Alice Chicken,...,Whole Wheat and Honey Pizza Dough,World's Best Lasagna,Yakisoba Chicken,Yummy Honey Chicken Kabobs,Yummy Pork Chops,Zesty Slow Cooker Chicken Barbecue,Zippy Summer Shrimp,Zucchini Alfredo,Zucchini Parmesan,Zucchini Patties
user_id,,,,,,,,,,,,,,,,,,,,,
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.25
559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.227848,NaN,NaN,NaN,NaN
1155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.029412,NaN,NaN,NaN,-0.970588,NaN,NaN,NaN,NaN
1299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Identify similar users

### Pearson correlation coefficient

In [25]:

pearson_sim = norm_ratings_matrix.T.corr(method='pearson')
pearson_sim.head()

user_id,455,559,833,1155,1299,1381,1537,1646,1873,2020,...,84374,84543,84570,84572,84583,84767,84778,84780,84790,84839
user_id,,,,,,,,,,,,,,,,,,,,,
455,1.000,1.0,-0.375000,-1.000000,-0.400000,NaN,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,-0.327327,NaN,NaN,NaN,-5.000000e-01,NaN,0.333333
559,1.000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,-1.000000
833,-0.375,NaN,1.000000,-0.461538,0.408248,NaN,NaN,-0.612372,0.57735,0.229416,...,NaN,-0.2,NaN,-0.258199,NaN,NaN,NaN,3.611576e-01,NaN,0.036274
1155,-1.000,NaN,-0.461538,1.000000,NaN,0.5,NaN,NaN,NaN,1.000000,...,NaN,-1.0,NaN,NaN,NaN,0.5,NaN,-1.000000e+00,-1.0,0.226455
1299,-0.400,NaN,0.408248,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,-9.614813e-17,NaN,NaN


In [58]:
num_sim_users = 10
sim_threshold = 0.4     # Noe her er fucka når jeg endrer dette tallet men aner ikke hvorfor
top_recipes = 10

def recommend_using_pearson(userID):     # Can add sim_matrix as parameter and be used for both pearson and cosine?? 
    # Making copy of the dataframe, so we dont change the original one
    
    sim_matrix = pearson_sim.copy()

    # remove userID from similarity matrix
    sim_matrix.drop(index = userID, inplace = True)

    # Top num_sim_users 
    sim_users = sim_matrix[sim_matrix[userID]>sim_threshold][userID].sort_values(ascending=False)[:num_sim_users]
    print(f'The similar users for user {userID} are', sim_users)

    # Getting only the recipes that our similar users have rated. 
    sim_user_recipes = norm_ratings_matrix[norm_ratings_matrix.index.isin(sim_users.index)].dropna(axis=1, how='all')

    # All the recipes the given user has rated, drops these as we do not want to recommend them
    userID_rated = norm_ratings_matrix[norm_ratings_matrix.index == userID].dropna(axis=1, how= 'all')
    sim_user_recipes.drop(userID_rated.columns,axis=1, inplace= True, errors = 'ignore')

    item_scores = {}   #dict to store item scores

    for i in sim_user_recipes.columns:
        recipe_rating = sim_user_recipes[i]
        total_score = 0
        num_scores = 0
        for j in sim_users.index:
            if pd.isna(recipe_rating[j]) == False:
                sum_score_rating = sim_users[j] * recipe_rating[j] 
                total_score += sum_score_rating
                num_scores+=1
        item_scores[i] = total_score/num_scores

    #turning dict into dataframe
    item_scores = pd.DataFrame(item_scores.items(),columns=['recipe', 'recipe_score'])
    
    ranking_recipe_scores = item_scores.sort_values(by='recipe_score', ascending=False)

    recipes_to_recommend = ranking_recipe_scores.head(top_recipes)
    
    return recipes_to_recommend


In [59]:
def recommend_and_predict(userID, sim_matrix):
    ranking_recipe_scores = recommend_using_pearson(userID, sim_matrix)

    # The average rating for the given user
    avg_rating = ratings_matrix[ratings_matrix.index == userID].T.mean()[userID]

    ranking_recipe_scores['pred_rating'] = ranking_recipe_scores['recipe_score'] + avg_rating
    recipes_to_predict = ranking_recipe_scores.head(top_recipes)

    return recipes_to_predict

In [60]:
pearson_copy = pearson_sim.copy()
recommend_and_predict(1155, pearson_copy)

The similar users for user 1155 are user_id
49957    1.0
50689    1.0
28280    1.0
48869    1.0
49555    1.0
26092    1.0
25963    1.0
66074    1.0
24721    1.0
23885    1.0
Name: 1155, dtype: float64


,recipe,recipe_score,pred_rating
165,Pork Chops Stuffed with Smoked Gouda and Bacon,1.350000,5.320588
105,Firecracker Grilled Alaska Salmon,1.350000,5.320588
192,Shrimp Fettuccine Alfredo,1.090909,5.061497
99,Easy Tuna Casserole,1.090909,5.061497
101,Eggplant Parmesan II,1.090909,5.061497
111,Garlic Prime Rib,1.090909,5.061497
191,Shepherd's Pie VI,0.954545,4.925134
244,The Best Meatloaf,0.954545,4.925134
92,Dee's Roast Pork for Tacos,0.954545,4.925134
237,Tacos in Pasta Shells,0.954545,4.925134


### Cosine similarity

Because cosine similarity does not take missing values (NaN), we must convert all NaN´s to zeros.

In [61]:
matrix_dummy = norm_ratings_matrix.copy().fillna(0)
cosine_simiarity_ = cosine_similarity(matrix_dummy, matrix_dummy)

#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_simiarity_, index=norm_ratings_matrix.index, columns=norm_ratings_matrix.index)

cosine_sim.head(10)

user_id,455,559,833,1155,1299,1381,1537,1646,1873,2020,...,84374,84543,84570,84572,84583,84767,84778,84780,84790,84839
user_id,,,,,,,,,,,,,,,,,,,,,
455,1.000000,0.036559,-0.021651,-0.021935,-0.035414,-0.008890,0.030601,0.000000,0.000000,0.145863,...,-0.036435,0.004534,0.032791,-0.022519,0.027190,-0.009465,0.020536,-0.025851,0.021385,-0.024355
559,0.036559,1.000000,-0.018876,0.000000,-0.180098,0.008357,0.000000,0.012797,0.000000,0.000000,...,0.000000,0.010959,0.013211,-0.038633,0.000000,-0.047522,0.008274,0.000000,0.034464,-0.085561
833,-0.021651,-0.018876,1.000000,-0.118334,-0.024201,0.005301,-0.001976,-0.046177,0.095624,0.070376,...,0.020950,-0.010234,-0.004190,-0.022462,-0.023162,-0.112879,-0.004198,0.058900,0.018217,0.003930
1155,-0.021935,0.000000,-0.118334,1.000000,0.004350,0.031434,0.035763,0.000000,-0.018245,0.059711,...,0.004747,-0.013087,0.000863,0.000688,-0.023618,0.023343,-0.015135,-0.021774,-0.051787,0.023323
1299,-0.035414,-0.180098,-0.024201,0.004350,1.000000,0.018470,-0.038146,0.036296,0.000000,0.054549,...,0.011355,0.000000,0.000000,0.012554,0.000000,0.002682,0.000000,-0.010063,-0.028274,-0.000511
1381,-0.008890,0.008357,0.005301,0.031434,0.018470,1.000000,-0.013058,0.000000,0.008714,-0.046474,...,-0.004535,-0.004836,-0.020406,-0.009299,0.011280,0.023560,-0.013631,0.000000,-0.021968,-0.020850
1537,0.030601,0.000000,-0.001976,0.035763,-0.038146,-0.013058,1.000000,0.000000,0.000000,0.000000,...,0.015421,0.000000,-0.005070,0.017184,-0.004204,0.032927,0.029688,0.000000,0.084865,-0.006276
1646,0.000000,0.012797,-0.046177,0.000000,0.036296,0.000000,0.000000,1.000000,0.000000,-0.019570,...,-0.030552,-0.037152,0.020831,-0.078357,0.017273,0.000000,0.000000,0.066088,0.018113,-0.001817
1873,0.000000,0.000000,0.095624,-0.018245,0.000000,0.008714,0.000000,0.000000,1.000000,0.020588,...,-0.002296,0.000000,-0.027550,0.005493,0.000000,0.000000,0.000000,-0.005297,0.005989,0.000000


In [63]:
# Ikke ferdig

'''
def recommend_using_cosine(userID):     # Can add sim_matrix as parameter and be used for both pearson and cosine?? 
    # Making copy of the dataframe, so we dont change the original one
    
    sim_matrix = cosine_sim.copy()

    # remove userID from similarity matrix
    sim_matrix.drop(index = userID, inplace = True)

    # Top num_sim_users 
    sim_users = sim_matrix[userID].sort_values(by='user_id',ascending=False)[:num_sim_users]
    sim_score_sum = sim_users.sum()

    print(f'The similar users for user {userID} are', sim_users)

    # Getting only the recipes that our similar users have rated. 
    sim_user_recipes = norm_ratings_matrix[norm_ratings_matrix.index.isin(sim_users.index)].dropna(axis=1, how='all')

    # All the recipes the given user has rated, drops these as we do not want to recommend them
    userID_rated = norm_ratings_matrix[norm_ratings_matrix.index == userID].dropna(axis=1, how= 'all')
    sim_user_recipes.drop(userID_rated.columns,axis=1, inplace= True, errors = 'ignore')

    item_scores = {}   #dict to store item scores

    for i in sim_user_recipes.columns:
        recipe_rating = sim_user_recipes[i]
        total_score = 0
        num_scores = 0
        for j in sim_users.index:
            if pd.isna(recipe_rating[j]) == False:
                sum_score_rating = sim_users[j] * recipe_rating[j] 
                total_score += sum_score_rating
                num_scores+=1
        item_scores[i] = total_score/num_scores

    #turning dict into dataframe
    item_scores = pd.DataFrame(item_scores.items(),columns=['recipe', 'recipe_score'])
    
    ranking_recipe_scores = item_scores.sort_values(by='recipe_score', ascending=False)

    recipes_to_recommend = ranking_recipe_scores.head(top_recipes)
    
    return recipes_to_recommend'''

### -----Vanessa sine endringer bare over denne linjen-----

In [ ]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

In [ ]:

X = df.copy()
y = df['user_id']

print(y)

In [ ]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y,random_state=42)

In [ ]:

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# Define the baseline model to always return 3.
def baseline(user_id, item_id):
    return 3.0

In [ ]:
def score(cf_model):
    # List of user-item tuples from test set
    id_pairs = zip(X_test['user_id'], X_test['item_id'])

    # Predict rating for every user-item tuple
    y_pred = np.array([cf_model(user, item) for (user, item) in id_pairs])
    print(y_pred)

    #Get actual ratings
    y_true = np.array(X_test['rating'])
    print('true ratings: ')
    print(y_true)
    # Final RMSE score
    return rmse(y_true, y_pred)



In [ ]:
score(baseline)

In [ ]:
# BUILDING RATINGS MATRIX

ratings_matrix = X_train.pivot_table(values='rating', index='user_id', columns='item_id')

ratings_matrix.head()

In [ ]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(user_id, item_id):
    
    #Check if movie_id exists in r_matrix
    if item_id in ratings_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = ratings_matrix[item_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [ ]:
score(cf_user_mean)

In [ ]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = ratings_matrix.copy().fillna(0)

In [ ]:

#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [ ]:
#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=ratings_matrix.index, columns=ratings_matrix.index)

cosine_sim.head(10)

In [ ]:
ratings_matrix.head()

In [ ]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(user_id, item_id):
    
    #Check if movie_id exists in r_matrix
    if item_id in ratings_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[user_id]
        
        #Get the user ratings for the movie in question
        m_ratings = ratings_matrix[item_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        
        simScore = sim_scores.sum()
        if simScore == 0:
                simScore = 1
        wmean_rating = np.dot(sim_scores, m_ratings) / simScore
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating

In [ ]:
score(cf_user_wmean)